In [1]:
#import dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests
import time
import re


In [2]:
#Executable Path
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA-Mars News

In [3]:
#visit url
nasa_url='https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(nasa_url)

In [4]:
#HTML object
nasa_html=browser.html

#Parse HTMl w. BeautifulSoup
nasa_soup=BeautifulSoup(nasa_html,'html.parser')

#Find latest article w. Title and paragraph
slide=nasa_soup.find('li',class_='slide')
result=slide.find_all('div', class_="content_title")
news_title=result[0].get_text()

news_p=nasa_soup.find('div',class_='article_teaser_body').text

#display scapped info
print(news_title)
print(news_p)

NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound
The agency's new Mars rover is put through a series of tests in vacuum chambers, acoustic chambers and more to get ready for the Red Planet.


# JPL Mars Space Images - Featured Image

In [5]:
#visit JPL and scrape featured image
jpl_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
expand=browser.find_by_css('a.fancybox-expand')
expand.click()
time.sleep(1)

jpl_html=browser.html
jpl_soup=BeautifulSoup(jpl_html,'html.parser')

img_relative=jpl_soup.find('img',class_='fancybox-image')['src']
image_path=f"https://www.jpl.nasa.gov{img_relative}"

print(image_path)
                                                                                           

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA15253_ip.jpg


# Mars Weather

In [6]:
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
time.sleep(1)

soup=BeautifulSoup(browser.html,'html.parser')

mars_weather=soup.find_all(text=re.compile('InSight'))[0]
print(mars_weather)

InSight sol 524 (2020-05-18) low -92.5ºC (-134.4ºF) high 0.5ºC (32.9ºF)
winds from the SW at 4.8 m/s (10.7 mph) gusting to 15.3 m/s (34.3 mph)
pressure at 7.00 hPa


# Mars-Facts

In [7]:
#use pandas to gather data
tables=pd.read_html('https://space-facts.com/mars/')

df= tables[0]
df=df.rename(columns={0:'Describe',1:'Value'})

df
                     

,Describe,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [8]:
#Convert table to HTML
mars_facts_html=[df.to_html(classes='data table table-borderless', index=False, header=False, border=0)]

mars_facts_html

['<table border="0" class="dataframe data table table-borderless">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>']

# Mars-Hemisphere

In [9]:
# scrape images of Mars' hemispheres from the USGS site
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(mars_hemisphere_url)
hemi_soup=BeautifulSoup(browser.html,"html.parser")

links=hemi_soup.find_all('h3')

hemisphere_img_urls=[]

for i in range(len(links)):
    hemi_dict ={}
    browser.find_by_tag('h3')[i].click()
    time.sleep(1)
    soup = BeautifulSoup(browser.html, 'html.parser')
    initial =soup.find('h2', class_='title')
    title = initial.text
    hemi_dict['title'] = title
    img_url = soup.select_one('img.wide-image').get('src')
    hemi_dict['img_url'] = img_url
    browser.back()
    time.sleep(1)
    hemisphere_img_urls.append(hemi_dict)

hemisphere_img_urls


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': '/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': '/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': '/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': '/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]